In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("dataset_without.csv")

In [6]:
df=df.drop(columns=['Unnamed: 0'])

In [8]:
# Convert float rating to int class label (0-indexed for BERT classification)
df['label'] = df['Rating'].astype(int) - 1  # Ratings 1–5 -> Labels 0–4

# Remove original Rating column
df = df[['Review', 'label']].rename(columns={'Review': 'text'})

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [10]:
from transformers import AutoTokenizer

#model_name = "microsoft/deberta-v3-small"
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/119640 [00:00<?, ? examples/s]

Map:   0%|          | 0/29911 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

training_args = TrainingArguments(
    output_dir="./roberta-review-classifier2",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",
    report_to="none",
    fp16=True
)


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



/tmp/ipython-input-3333095094.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,1.551000
100,1.362800
150,1.128500
200,1.029000
250,0.985200
300,1.035700
350,0.956000
400,0.975700
450,0.956100
500,0.941400


TrainOutput(global_step=14956, training_loss=0.8053676724338251, metrics={'train_runtime': 1549.1512, 'train_samples_per_second': 154.459, 'train_steps_per_second': 9.654, 'total_flos': 7869863641098240.0, 'train_loss': 0.8053676724338251, 'epoch': 2.0})

In [14]:
trainer.save_model('./epoch 2')

In [15]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 2:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 2:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 2: 0.7179120695419592
 Training F1 after Epoch 2: 0.7106972488064143


In [16]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 2:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 2:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 2: 0.676506970679683
 Eval F1 after Epoch 2: 0.6690164657241466


In [17]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 2')

In [18]:

training_args = TrainingArguments(
    output_dir='./roberta-review-classifier2',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",

    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Resume training from checkpoint
trainer.train(resume_from_checkpoint=True)


/tmp/ipython-input-731694944.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
15000,0.712800
15050,0.746100
15100,0.740100
15150,0.720700
15200,0.741500
15250,0.713500
15300,0.768500
15350,0.707800
15400,0.680000
15450,0.748500


TrainOutput(global_step=29912, training_loss=0.3493801494370364, metrics={'train_runtime': 1554.1031, 'train_samples_per_second': 307.933, 'train_steps_per_second': 19.247, 'total_flos': 1.573972728219648e+16, 'train_loss': 0.3493801494370364, 'epoch': 4.0})

In [19]:
trainer.save_model('./epoch 4')

In [20]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 4:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 4:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 4: 0.7534687395519893
 Training F1 after Epoch 4: 0.7486517482102788


In [21]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 4:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 4:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 4: 0.6761392129985624
 Eval F1 after Epoch 4: 0.6707851553055838


In [22]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 4')

In [23]:

training_args = TrainingArguments(
    output_dir='./roberta-review-classifier2',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",

    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Resume training from checkpoint
trainer.train(resume_from_checkpoint=True)


/tmp/ipython-input-1275749964.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
29950,0.668400
30000,0.605400
30050,0.689900
30100,0.712000
30150,0.674900
30200,0.679900
30250,0.693800
30300,0.627200
30350,0.635700
30400,0.641700


TrainOutput(global_step=44868, training_loss=0.2154489239875227, metrics={'train_runtime': 1567.3167, 'train_samples_per_second': 458.006, 'train_steps_per_second': 28.627, 'total_flos': 2.360959092329472e+16, 'train_loss': 0.2154489239875227, 'epoch': 6.0})

In [24]:
trainer.save_model('./epoch 6')

In [25]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 6:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 6:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 6: 0.7786108324974925
 Training F1 after Epoch 6: 0.774270977207487


In [26]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 6:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 6:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 6: 0.6741666945270971
 Eval F1 after Epoch 6: 0.6689672448283313
